# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#store filepath in variable
vacation = ("../output_data/cityweather.csv")

#read csv file
vacation_df = pd.read_csv(vacation, encoding="ISO-8859-1")

#show header
vacation_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,qaanaaq,95,GL,1569453939,76,77.48,-69.36,18.61,10.38
1,moerai,95,PF,1569453940,83,-22.43,-151.33,72.66,11.07
2,mataura,100,NZ,1569453778,88,-46.19,168.86,46.99,18.99
3,umm kaddadah,77,SD,1569453941,20,13.60,26.69,90.10,8.88
4,avarua,99,CK,1569453766,78,-21.21,-159.78,77.00,13.87


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# set up latitude and longitude
locations = vacation_df[['Lat','Lng']]

# capture humidity for weighting
humid = vacation_df['Humidity'].astype(float)


In [4]:
# Plot Heatmap
# Customize the size of the figure
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

fig = gmaps.figure(layout=figure_layout)

fig = gmaps.figure(map_type='SATELLITE')

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humid, 
                                 dissipating=False, max_intensity=50,
                                 point_radius=2)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
#store filepath in variable
weather = ("../output_data/cityweather.csv")

#read csv file
weather_df = pd.read_csv(weather, encoding="ISO-8859-1")

#set up to filter on weather conditions
weather_df = weather_df[(weather_df['Max Temp'] > 70) & (weather_df['Max Temp'] < 80)]
weather_df = weather_df[(weather_df['Wind Speed'] < 10)]
weather_df = weather_df[(weather_df['Cloudiness'] == 0)]

weather_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
29,rikitea,0,PF,1569453948,64,-23.12,-134.97,72.05,4.07
41,gizo,0,IL,1569453952,73,31.80,34.94,73.40,5.82
104,sur,0,OM,1569453968,74,22.57,59.53,78.75,7.76
108,cabedelo,0,BR,1569453970,83,-6.97,-34.84,73.40,6.93
217,abha,0,SA,1569453862,37,18.22,42.50,71.60,4.70


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
#store df in new name hotel_df
hotel_df = weather_df

# add Hotel Name to dataframe as well as hotel lat and lng
hotel_df["Hotel Name"]=""
hotel_df["Hotel Lat"]=""
hotel_df["Hotel Lng"]=""
hotel_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name,Hotel Lat,Hotel Lng
29,rikitea,0,PF,1569453948,64,-23.12,-134.97,72.05,4.07,,,
41,gizo,0,IL,1569453952,73,31.80,34.94,73.40,5.82,,,
104,sur,0,OM,1569453968,74,22.57,59.53,78.75,7.76,,,
108,cabedelo,0,BR,1569453970,83,-6.97,-34.84,73.40,6.93,,,
217,abha,0,SA,1569453862,37,18.22,42.50,71.60,4.70,,,


In [7]:
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

# Use the lat/lng to identify hotels
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    # URL
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request
    hotel_name = requests.get(base_url, params=params)

    # convert to json
    name_hotel = hotel_name.json()
    
    hotel_df.loc[index, "Hotel Name"] = name_hotel["results"][0]["name"]
    hotel_df.loc[index, "Hotel Lat"] = name_hotel["results"][0]["geometry"]["location"]["lat"]
    hotel_df.loc[index, "Hotel Lng"] = name_hotel["results"][0]["geometry"]["location"]["lng"]
    
    print(json.dumps(name_hotel, indent=4, sort_keys=True))

{
    "html_attributions": [],
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": -23.1206197,
                    "lng": -134.9687529
                },
                "viewport": {
                    "northeast": {
                        "lat": -23.1194598197085,
                        "lng": -134.9673886697085
                    },
                    "southwest": {
                        "lat": -23.1221577802915,
                        "lng": -134.9700866302915
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png",
            "id": "60864d02c84a997a4e41d2ee16ce4b24b3e67a6a",
            "name": "Pension Maro'i",
            "opening_hours": {
                "open_now": true
            },
            "photos": [
                {
                    "height": 2304,
                    "html_attributions": [
                     

{
    "html_attributions": [],
    "next_page_token": "CqQCHgEAAIHRnHsS3qyKM0F8fLBryyOr7TAq_rmjQCmQHBYayya3ephC8QFzR9iEx5Ehx6pDQ3rQPQ5N00RgLP5ftKCk1OGc0XYfy2IeanLBsOqPP7pSleHP_BkdRfA8ZfcX_-8xhg0ChjPZmx7RnIwncEDgAXLMip0zcasP45IYeN5CJK49xREr-6ZwaP7-C5olLio6FsSHQtfQAFPsLPhS_e9KqN_C2e9d_NfmHUtWQyyL1xJNXC65tinUOxEQ5Sy_O26e1HaMDjIRHY0bif4sX8mpgBgWPURLNeOenwIB48UDK3sPn59aArpuoAODODS3D-45N-uOKeGODx3eyaWUp0YVOjcKuDhSsMpsrPLYoB5TgVyy8vZlKWUp6fRbX_JQMenVXhIQjp9RiOiLUvIEsoze58VfMhoUXh_mgg-2oTB-1w5WeebH9TZUN3U",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 22.5664354,
                    "lng": 59.4958585
                },
                "viewport": {
                    "northeast": {
                        "lat": 22.5678508302915,
                        "lng": 59.49709478029151
                    },
                    "southwest": {
                        "lat": 22.5651528697085,
                        "lng": 59.494396819708

{
    "html_attributions": [],
    "next_page_token": "CqQCHgEAAG-TWKfhDjc2l579LSmA4DAfzyqZCAK1FWXpkHWH0vMnZBUWjarOIS6bqEOOoZk_XkBCKK9aG3lODmQe4y2Fiooqz57U7syjQTH0Y-7QbZ2pDxLzdthLJc8AL68YUPRefucAs9Ws6qP_RPWSrqmRy8AB-RxVU4TaiUGz6pCmaw82me_SiflfQuW0OCq9qFjrFuSSGFTWyoIGb9y4b-fZQPX4QLKVy0kfM4AJpxPoORLLw8IHZcaWCqsYFkaPwr5cJDmQOe6Zr1vo0tyPdXivM_7OrW1RZeePAmYIlAt8AzTkLe2seaVRR4FBhd6ZX18FAdJfa8xFiDmYIZIguOi32vD1IdyOnmVWtL3RaZfDXEqypsmvdVheuhFmsItxHIzHmRIQOSJqAHm6pKrRUkvpZBRzKxoU1puXTRE3JfBQYos5CfZklCPz0os",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": -6.9814516,
                    "lng": -34.8289625
                },
                "viewport": {
                    "northeast": {
                        "lat": -6.980105769708498,
                        "lng": -34.8276379197085
                    },
                    "southwest": {
                        "lat": -6.982803730291502,
                        "lng": -34.830335

{
    "html_attributions": [],
    "next_page_token": "CqQCHgEAAFCJddRH7anoX5OI4V_6A2T9QYCkhwiBBnous7B0jHGBq6dxx_o0RwCXbo3IRWKd-vLjZmGkd_zxfKV1YebnXn7Ufqjsa-aHu8co-i1Z1l-iPAal7SUY4V1HD44oNHmU_RO-YEXf4kQq3CQnMmQQNikBRBtwNFNzC2Ng-UO5wwJQtIWTi0lqADJtcUAk0xhkbi2f2AXnP3sn4JEW4cVkzT4TDEIj4s_tpgv5s-HoBTxq7HaLt4FeQtPJJR7Tu8GMWkIiWHJTsovd2mdge_qXPeRfnHlbYLrI2jQg9q_4hsAEwzG36qwJByN7aeAloA-FiY2hCC5vkjeO980F66l3GTDDFxeQyv0C2oFDYUv6M4b5XQLDWhBD5OgQcV9dtlfB5xIQ_wbq4Ot3XFoeaX4QvPQjVRoUI6BfHQF6VHi29oEi-D3Y9p597qU",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 18.2133236,
                    "lng": 42.4998911
                },
                "viewport": {
                    "northeast": {
                        "lat": 18.2146190302915,
                        "lng": 42.5011862302915
                    },
                    "southwest": {
                        "lat": 18.2119210697085,
                        "lng": 42.4984882697085

{
    "html_attributions": [],
    "next_page_token": "CqQCHgEAADU_qSKSdONzFvYDbcbnY9gNfCsO_HZEZIbZe-3g5Yonpz9ZbUfwxBB3zX5EVGIJCsRkP4LcotrBnhgWGL71-MJ33NhOPwIu60Pnt5wCyrroeGf6SFPGW7Bcn5xc8eJlSVU7qkmy3bAOHqWnj0d7-qi1AUVEHA7QWhCvLBcdeMkAceYYfrV0mK2q0lAUO31dx3PqeP7QGYJ-Dg8uELzrsT7CGVoXCio_VM-UYe_HnClShzSpbnWqYqIS21v9kgrKK0B6ALWgWiwoyJEipGiTzw_GOPrsWKJHiyXKlR8tp-GXrJaz5SvKC8vlGykqMLH6i1JTwYMi_oISuuYOtPVh7nH90jaPO2MoNzYRmBfCkP9-Jr5B1OrMd107mAfK6CuSjRIQKpO2zjYvIFa4zffKbxxVRhoUW_3-dCbskOXiEO1WS3iaCer7axM",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": -34.443902,
                    "lng": -58.890839
                },
                "viewport": {
                    "northeast": {
                        "lat": -34.44305951970851,
                        "lng": -58.88987384999999
                    },
                    "southwest": {
                        "lat": -34.4457574802915,
                        "lng": -58.8935796

...

{
    "html_attributions": [],
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": -31.6548849,
                    "lng": -68.39364499999999
                },
                "viewport": {
                    "northeast": {
                        "lat": -31.6535364697085,
                        "lng": -68.39253521970849
                    },
                    "southwest": {
                        "lat": -31.6562344302915,
                        "lng": -68.3952331802915
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png",
            "id": "de16b2dba136d40600ac484ee0065da14fadc52e",
            "name": "Gabriela rocio gomez",
            "opening_hours": {
                "open_now": false
            },
            "place_id": "ChIJHYEkqPERgZYREcPNyEQcF1c",
            "plus_code": {
                "compound_code": "8JW4+2G Nueve 

{
    "html_attributions": [],
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 33.1397452,
                    "lng": 11.2171144
                },
                "viewport": {
                    "northeast": {
                        "lat": 33.1410897802915,
                        "lng": 11.2184677802915
                    },
                    "southwest": {
                        "lat": 33.1383918197085,
                        "lng": 11.2157698197085
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png",
            "id": "c6175800d67ce87b3dd7b18c4019866ffb604f9b",
            "name": "hotel rayes",
            "photos": [
                {
                    "height": 653,
                    "html_attributions": [
                        "<a href=\"https://maps.google.com/maps/contrib/111389563663237387634/photos\">Ilef M</a>

{
    "html_attributions": [],
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 31.20643780000001,
                    "lng": 16.5889102
                },
                "viewport": {
                    "northeast": {
                        "lat": 31.2075490802915,
                        "lng": 16.59023378029151
                    },
                    "southwest": {
                        "lat": 31.2048511197085,
                        "lng": 16.5875358197085
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png",
            "id": "a0bbf600a119257d51512965041aa03d6672c3d6",
            "name": "City Hotel",
            "place_id": "ChIJ7xjCY-R-mBMRsFq8exbIYMA",
            "plus_code": {
                "compound_code": "6H4Q+HH Sirte, Libya",
                "global_code": "8F3R6H4Q+HH"
            },
            "rating": 3.8,


{
    "html_attributions": [],
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": -8.519586699999998,
                    "lng": 120.9998365
                },
                "viewport": {
                    "northeast": {
                        "lat": -8.518231219708497,
                        "lng": 121.0014788802915
                    },
                    "southwest": {
                        "lat": -8.520929180291501,
                        "lng": 120.9987809197085
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png",
            "id": "c947b6514c0416efb56a5c0b6654e0ca0185896d",
            "name": "Ellfors",
            "place_id": "ChIJN1Smkff5si0R1KoVo8UmHOM",
            "plus_code": {
                "compound_code": "FXJX+5W Riung, Central Benteng, Ngada Regency, East Nusa Tenggara, Indonesia",
                "global_cod

{
    "html_attributions": [],
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 30.7521952,
                    "lng": 20.2143234
                },
                "viewport": {
                    "northeast": {
                        "lat": 30.7535196302915,
                        "lng": 20.2157168302915
                    },
                    "southwest": {
                        "lat": 30.7508216697085,
                        "lng": 20.2130188697085
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png",
            "id": "2e87d1e262737d8d8c8faa4c935a2ba2575d7f92",
            "name": "\u0633\u0639\u064a\u062f \u0627\u0644\u0635\u0628\u062d\u064a \u0644\u0644\u062a\u0643\u064a\u064a\u0641 \u0648\u0627\u0644\u062a\u0628\u0631\u064a\u062f",
            "place_id": "ChIJ_V7swAH9hBMRgIt4M7wzg8s",
            "plus_code": {
         

In [7]:
# Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
hotel_df.head()


,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name,Hotel Lat,Hotel Lng
29,rikitea,0,PF,1569453948,64,-23.12,-134.97,72.05,4.07,,,
41,gizo,0,IL,1569453952,73,31.80,34.94,73.40,5.82,,,
104,sur,0,OM,1569453968,74,22.57,59.53,78.75,7.76,,,
108,cabedelo,0,BR,1569453970,83,-6.97,-34.84,73.40,6.93,,,
217,abha,0,SA,1569453862,37,18.22,42.50,71.60,4.70,,,


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]
# print(hotel_info)

In [9]:
# Add marker layer ontop of heat map
# Display Map
markers = gmaps.marker_layer(locations)

fig.add_layer(markers)

fig

Figure(layout=FigureLayout(height='420px'))